In [1]:
import pyspark
import numpy as np
from DataUtils import DataUtil

sc = pyspark.SparkContext('local[*]')

In [24]:
dataUtil = DataUtil(sc, 'data/spam.data.txt', 'data/mean_std.txt', True)
rdd = dataUtil.read(sc)

In [25]:
rdd = rdd.map(lambda x: (1, x[0], x[1]))
w = (0, np.zeros(56))

rdd = rdd.map(lambda x: (x[0], x[1], x[2], sigmoid(w[1].dot(x[1]) + w[0] * x[0])))


In [22]:
temp = rdd.map(lambda x: ((x[3] - x[2]) * x[0], (x[3] - x[2]) * x[1])) \
                         .reduce(lambda a,b: (a[0] + b[0], a[1] + b[1]))

In [27]:
dw = (dw[0]/4601, (dw[1]/4601) + (0.01/4601)*w[1])

In [28]:
dw

(0.10595522712453814,
 array([-0.06166228,  0.01476707, -0.09624795, -0.0280302 , -0.1182002 ,
        -0.11365021, -0.16227089, -0.10104403, -0.11313222, -0.06789469,
        -0.11459167, -0.00378183, -0.0649461 , -0.02932905, -0.09571774,
        -0.12860496, -0.12859872, -0.09977388, -0.13370225, -0.0927151 ,
        -0.18724621, -0.04487992, -0.16357383, -0.105589  ,  0.1254366 ,
         0.11382585,  0.08960956,  0.07758878,  0.06523799,  0.083596  ,
         0.06200866,  0.05580396,  0.05859695,  0.05509003,  0.07291052,
         0.06651443,  0.08699248,  0.01516386,  0.06001418,  0.03166087,
         0.04757701,  0.06674886,  0.06628478,  0.04621748,  0.06860189,
         0.07140225,  0.0218296 ,  0.04105208,  0.02913455,  0.04381238,
         0.03161751, -0.11818471, -0.15812369, -0.03179115, -0.05374535,
        -0.10558382]))

In [29]:
w = (w[0] - 0.01 * dw[0], w[1] - 0.01 * dw[1])

In [30]:
w

(-0.0010595522712453815,
 array([ 6.16622830e-04, -1.47670727e-04,  9.62479485e-04,  2.80301998e-04,
         1.18200201e-03,  1.13650206e-03,  1.62270888e-03,  1.01044027e-03,
         1.13132224e-03,  6.78946933e-04,  1.14591672e-03,  3.78182945e-05,
         6.49461025e-04,  2.93290461e-04,  9.57177356e-04,  1.28604960e-03,
         1.28598720e-03,  9.97738815e-04,  1.33702247e-03,  9.27150966e-04,
         1.87246212e-03,  4.48799160e-04,  1.63573832e-03,  1.05588995e-03,
        -1.25436605e-03, -1.13825850e-03, -8.96095638e-04, -7.75887770e-04,
        -6.52379935e-04, -8.35959955e-04, -6.20086607e-04, -5.58039595e-04,
        -5.85969452e-04, -5.50900302e-04, -7.29105168e-04, -6.65144296e-04,
        -8.69924780e-04, -1.51638603e-04, -6.00141817e-04, -3.16608680e-04,
        -4.75770094e-04, -6.67488617e-04, -6.62847780e-04, -4.62174807e-04,
        -6.86018895e-04, -7.14022526e-04, -2.18295972e-04, -4.10520822e-04,
        -2.91345547e-04, -4.38123805e-04, -3.16175101e-04,  1.1

In [31]:
rdd.count()

4601

In [ ]:
rdd = rdd.map(lambda x: (np.insert(x[0], 0, 1), x[1]))
w = np.zeros(57)

In [ ]:
rdd.first()

In [5]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
rdd = rdd.map(lambda x: (x[0], x[1], sigmoid(w.dot(x[0]))))

In [ ]:
rdd.first()

In [ ]:
np.full(57, -1)

In [ ]:
dw = rdd.map(lambda x: (x[2] - x[1]) * x[0]) \
        .reduce(lambda a,b: a + b)

In [ ]:
dw = (dw/4601) + (0.01/4601) * dw

In [ ]:
dw

In [ ]:
rdd.first()

In [35]:
class ParallelLogReg:
    
    def __init__(self, sc, dataUtils, iterations, learning_rate, lambda_reg, fit_intercept):
        self.dataUtils = dataUtils
        # do we need to broadcast these?
        self.iterations = iterations
        self.lr = learning_rate
        self.lambda_reg = lambda_reg
        self.fit_intercept = fit_intercept
        self.data = self.dataUtils.read(sc)
        self.numberObservations = self.data.count()
        self.numberFeatures = len(self.data.first()[0])
        
    def __add_intercept(self):
        self.data = self.data.map(lambda x: (1, x[0], x[1]))
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def __predict_y(self, w, x):
        return self.__sigmoid(w[1].dot(x[1]) + w[0] * x[0])
    
    def calculateLoss(self):
        return 1
        
    def train(self, sc):
        self.__add_intercept()
        
        # initialize the weights
        # w[0]: bias weight
        # w[1]: rest of the weights
        self.w = sc.broadcast((0, np.zeros(self.numberFeatures)))
        self.loss = 0
        
        # initialize prediction to rdd
        # x[0]: bias/intercept
        # x[1]: rest features
        # x[2]: true y
        # adding x[3]: predicted y
        self.data = self.data.map(lambda x: (x[0], x[1], x[2], self.__sigmoid(self.w[1].dot(x[1]) + self.w[0] * x[0])))

        for i in range(self.iterations):
            # compute derivatives
            self.dw = self.data.map(lambda x: ((x[3] - x[2]) * x[0], (x[3] - x[2]) * x[1])) \
                         .reduce(lambda a,b: (a[0] + b[0], a[1] + b[1]))
            self.dw = (dw[0]/self.numberObservations, (dw[1]/self.numberObservations) + (self.lambda_reg/self.numberObservations)*self.w[1])
            
            # update weights
            self.w = (self.w[0] - self.lr * self.dw[0], self.w[1] - self.lr * self.dw[1])
            
            # update prediction
            #self.data = self.__update_prediction(self.data, self.w)
            #self.loss = self.data.map(lambda x: x[2] * np.log(x[3]) + (1 - x[2]) * np.log(1 - x[3])) 
                                 #.reduce(lambda a,b: a + b)
            #self.loss = -(1/self.numberObservations) * self.loss + (self.lambda_reg/(2*self.numberObservations)) * np.sum(self.w**2)
            #print(self.data.first())
            
        return self.w
            

In [36]:
logReg = ParallelLogReg(sc, dataUtil, 1, 0.01, 0.01, True)

In [37]:
logReg.train(sc)

Traceback (most recent call last):
  File "/usr/local/spark/python/pyspark/serializers.py", line 587, in dumps
    return cloudpickle.dumps(obj, 2)
  File "/usr/local/spark/python/pyspark/cloudpickle.py", line 863, in dumps
    cp.dump(obj)
  File "/usr/local/spark/python/pyspark/cloudpickle.py", line 260, in dump
    return Pickler.dump(self, obj)
  File "/opt/conda/lib/python3.6/pickle.py", line 409, in dump
    self.save(obj)
  File "/opt/conda/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/opt/conda/lib/python3.6/pickle.py", line 751, in save_tuple
    save(element)
  File "/opt/conda/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/usr/local/spark/python/pyspark/cloudpickle.py", line 406, in save_function
    self.save_function_tuple(obj)
  File "/usr/local/spark/python/pyspark/cloudpickle.py", line 549, in save_function_tuple
    save(state)
  File "/opt/c

PicklingError: Could not serialize object: Exception: It appears that you are attempting to broadcast an RDD or reference an RDD from an action or transformation. RDD transformations and actions can only be invoked by the driver, not inside of other transformations; for example, rdd1.map(lambda x: rdd2.values.count() * x) is invalid because the values transformation and count action cannot be performed inside of the rdd1.map transformation. For more information, see SPARK-5063.

In [ ]:
logReg.data.first()

In [ ]:
class LogisticRegression:
    def __init__(self, lr=0.01, num_iter=100000, fit_intercept=True, verbose=False):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.verbose = verbose
    
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def __loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        # weights initialization
        self.theta = np.zeros(X.shape[1])
        
        for i in range(self.num_iter):
            z = np.dot(X, self.theta)
            h = self.__sigmoid(z)
            gradient = np.dot(X.T, (h - y)) / y.size
            self.theta -= self.lr * gradient
            
            if(self.verbose == True and i % 10000 == 0):
                z = np.dot(X, self.theta)
                h = self.__sigmoid(z)
                print('loss: {' + str(self.__loss(h, y)) + '} \t')
    
    def predict_prob(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
    
        return self.__sigmoid(np.dot(X, self.theta))
    
    def predict(self, X, threshold=0.5):
        return self.predict_prob(X) >= threshold